In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import requests
from bs4 import BeautifulSoup
import time

In [2]:
# 1건의 대화 내용을 저장하는 클래스 => 질문과 답변이 한 쌍으로 저장되는 클래스
class Conversation:
    def __init__(self, question, answer):
        self.question = question
        self.answer = answer
    def __str__(self):
        return f'질문: {self.question}\n답변: {self.answer}'

In [3]:
c = Conversation('너.. 누구야??', '글쎄... 알아서 뭐하시려고...')
print(c)

질문: 너.. 누구야??
답변: 글쎄... 알아서 뭐하시려고...


In [4]:
# 75개의 대화 주제와 세부 대화 내용의 url을 클롤링해서 리턴하는 함수
def getSubjects():
    targetSite = 'https://basicenglishspeaking.com/daily-english-conversation-topics/'
    req = requests.get(targetSite)
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    
    subjects = []
    contentLinks = [] # 세부 대화 내용의 url을 저장할 빈 리스트를 선언한다.
    divs = soup.findAll('div', {'class': 'tcb-flex-col'})
    for div in divs:
        chapter = div.findAll('a')
        for a in chapter:
            subjects.append(a.text)
            # 세부 대화 내용의 url을 contentLinks에 추가한다.
            contentLinks.append(a.get('href'))
    
    # 대화 주제와 대화 주제에 따른 세부 대화 내용의 url을 리턴시킨다.
    # 리턴되는 데이터가 2개가 아니라 subjects, contentLinks를 1개의 튜플로 묶어서 리턴시킨다.
    return subjects, contentLinks

In [14]:
subjects, contentLinks = getSubjects()
# for i in range(len(subjects)):
    # print('{:2d}. {}: {}'.format(i + 1, subjects[i], contentLinks[i]))
    
# 대화 주제에 따른 전체 대화 내용을 저장할 빈 리스트를 선언한다.
conversation = list() # Conversation 클래스 객체를 저장한다.

i = 0 # 대화 주제의 개수를 세는 변수
# 대화 주제의 개수만큼 반복하며 각각의 대화 주제에 따른 대화 내용을 읽어서 저장한다.
for subject in subjects[:]:
    conversation.append('{:2d}. {}: {}'.format(i + 1, subjects[i], contentLinks[i]))
    
    # 대화 주제별 대화 내용을 크롤링할 페이지를 요청한다.
    req = requests.get(contentLinks[i])
    
    # time 모듈을 import 하면 sleep() 메소드를 사용할 수 있다.
    # sleep() 메소드는 인수로 지정한 시간만큼 프로그램을 멈춘다. => 시간은 초 단위로 지정한다.
    # 대형 포털 사이트는 짧은 간격으로 많은 요청이 들어오면 자기네 사이트가 공격당하는 것으로 판단할 수 있기 때문에
    # 일정한 간격을 두고 정보를 요청한다.
    # 자동화된 크롤링을 할 경우 페이지가 로딩되는 시간만큼 기다리기 위해서 일정한 시간 간격을 두고 정보를 요청한다.
    time.sleep(1)
    
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    
    # 대화 주제에 따른 대화 내용은 플레이 버튼(class 속성이 'sc_player_container1'인 div 태그)의 다음 형제이다.
    buttons = soup.findAll('div', {'class': 'sc_player_container1'}) # 플레이 버튼 전체를 얻어온다.
    # print(len(divs)) # 플레이 버튼의 개수 => 크롤링할 문장의 개수
    
    # 각각의 대화 주제에 따른 문장의 개수만큼 반복한다.
    for button in buttons:
        # index() 메소드로 전체 중에서 인수로 지정한 객체의 index를 얻어올 수 있다.
        # buttons.index(button) => 전체 플레이 버튼(buttons)에서 특정 버튼(button)의 index를 얻어온다.
        # index() 메소드의 실행 결과가 짝수면 질문이고 홀수면 답변이다.
        # print(buttons.index(button))
        
        # 크롤링할 문장은 플레이 버튼 내부에 작성된 것이 아니고 다음 형제로 작성되어 있다.
        # next_sibling: 다음 형제, previous_sibling: 이전 형제
        if buttons.index(button) % 2 == 0: # 질문인가?
            # print('질문: {}'.format(button.next_sibling))
            question = button.next_sibling
        else:
            # print('답변: {}'.format(button.next_sibling))
            answer = button.next_sibling
            # 대화 1건이 완성된다.
        
            # Conversation 클래스 객체를 만드는 작업은 질문과 답변이 한 쌍이 되는 순간 실행한다.
            # 질문과 답변이 한 쌍이 되었으므로 Conversation 클래스 객체를 생성하고 질문과 답변을 저장한다.
            c = Conversation(question, answer)
            # 질문과 답변이 한 쌍으로 저장된 Conversation 클래스 객체를 대화 내용을 기억할 conversation
            # 리스트에 추가한다.
            conversation.append(c)
        # ===== if    
            
    # 대화 주제 1건이 처리 완료되면 다음 대화를 처리하기 위해서 i를 1증가시킨다.
    i += 1

In [15]:
for c in conversation:
    print(c)

 1. Family: https://basicenglishspeaking.com/family/
질문: How many people are there in your family?
답변: There are 5 people in my family: my father, mother, brother, sister, and me.
질문: Does your family live in a house or an apartment?
답변: We live in a house in the countryside.
질문: What does your father do?
답변: My father is a doctor. He works at the local hospital.
질문: How old is your mother?
답변: She is 40 years old, 1 year younger than my father.
질문: Do you have any siblings? What’s his/her name?
답변: Yes, I do. I have 1 elder brother, David, and 1 younger sister, Mary.
질문: Are you the oldest amongst your brothers and sisters?
답변: No, I’m not. I’m the second child in my family.
질문: What does your mother/father like?
답변: My father likes playing football and my mother likes cooking.
질문: Do your parents let you stay out late?
답변: Of course not. They always ask me to get home before 10 pm each night.
질문: Do you stay with your parents?
답변: Right now, no, but I used to.
질문: Does your family us